# ci-hsc Test

This notebook implements the commands in [DMTN-023](https://dmtn-023.lsst.io), modified to use PSF matched co-adds, on the ci_hsc repo to build images and catalogs to deblend

# Python Imports and initialization

In [ ]:
export CI_HSC_DIR=$LSST/code/ci_hsc
mkdir test_data
echo "lsst.obs.hsc.HscMapper" > test_data/_mapper
ingestImages.py test_data $CI_HSC_DIR/raw/*.fits --mode=link
cd test_data
ln -s $CI_HSC_DIR/CALIB .
mkdir ref_cats
cd ref_cats
ln -s $CI_HSC_DIR/ps1_pv3_3pi_20170110 .

## Exposure Processing

In [ ]:
singleFrameDriver.py $LSST/test_data --rerun example1 \
  --cores=4 --id visit=903334..903338:2 --id visit=903342..903346:2 \
  --id visit=903986..903990:2 --id visit=904010^904014

# Create Skymap

Make a skymap of a single tract for a single pointing

In [ ]:
makeDiscreteSkyMap.py $LSST/test_data --rerun example1:example2 \
  --id visit=903334..903338:2 --id visit=903342..903346:2 \
  --id visit=903986..903990:2 --id visit=904010^904014 \
  --config skyMap.projection="TAN"

# Find the FWHM of the desired PSF

Use the worst PSF in all of the coadds and use it's FWHM, then create a config file

In [106]:
import os
from lsst.daf.persistence import Butler

WORK_DIR = os.path.join("/tigress", "fred", "lsst", "test_data")
butler = Butler(inputs=os.path.join(WORK_DIR, "rerun/example1"))

In [109]:
fwhm = 0
for B in ["I", "R"]:
    visits = butler.queryMetadata('raw', ['visit'], dataId={'filter':'HSC-'+B})
    for visit in visits:
        #print(visit)
        ccd = None
        for n in range(117):
            if butler.datasetExists('calexp', dataId={'visit':visit, 'ccd': n}):
                ccd = n
                exposure = butler.get("calexp", visit=visit, ccd=ccd)
                _fwhm = exposure.getPsf().computeShape().getDeterminantRadius()*FwhmPerSigma
                #print("ccd {0} fwhm: {1:.2f}".format(ccd, _fwhm))
        if _fwhm > fwhm:
            fwhm = _fwhm
print("Matching codds with fwhm={0:.2f}".format(fwhm))
config_contents = "\n".join(["config.makeCoaddTempExp.makePsfMatched=True",
                            "config.makeCoaddTempExp.modelPsf.defaultFwhm={0:.2f}".format(fwhm),
                            "config.makeCoaddTempExp.doApplyUberCal=False",
                            "config.assembleCoadd.doApplyUberCal=False"])
with open(os.path.join(WORK_DIR, "config.txt"), 'w') as f:
    f.write(config_contents)

Matching codds with fwhm=8.04


# Create Coadds



### Run coaddDriver on a single patch (in each filter)
Note that these are the same as the scripts from DMTN-023 with the notable exception that we have removed the `--config` options and added a `configfile` option to point to the config file we just created, which makes the coadds PSF homogenized.

We perform the coadd on a single image first so that we can check that the psf matching works as expected, otherwise see [the documentation](https://github.com/lsst/pipe_tasks/blob/master/python/lsst/pipe/tasks/makeCoaddTempExp.py#L145-L203) for a guide to troubleshooting PSF coaddition and modify the previous cell to update the config parameters as needed

In [ ]:
coaddDriver.py $LSST/test_data --rerun example1:example2 \
  --selectId visit=903334..903338:2 --selectId visit=903342..903346:2 \
  --id tract=0 patch=1,1 filter=HSC-R --cores=8 \
  --configfile=$LSST/test_data/config.txt
coaddDriver.py $LSST/test_data --rerun example1:example2 \
  --selectId visit=903986..903990:2 --selectId visit=904010^904014 \
  --id tract=0 patch=1,1 filter=HSC-I --cores=8 \
  --configfile=$LSST/test_data/config.txt

### Once you have verified PSF matching is working properly, run the following scripts on the remaining patches

In [ ]:
coaddDriver.py $LSST/test_data --rerun example1:example2 \
  --selectId visit=903334..903338:2 --selectId visit=903342..903346:2 \
  --id tract=0 patch=0,0^0,1^0,2^1,0^1,1^1,2^2,0^2,1^2,2 filter=HSC-R \
  --cores=8 --configfile $LSST/test_data/config.txt
coaddDriver.py $LSST/test_data --rerun example1:example2 \
  --selectId visit=903986..903990:2 --selectId visit=904010^904014 \
  --id tract=0 patch=0,0^0,1^0,2^1,0^1,1^1,2^2,0^2,1^2,2 filter=HSC-I \
  --cores=8 --configfile $LSST/test_data/config.txt

### Now we merge the catalogs on the PSF Matched Co-Adds

In [ ]:
mergeCoaddDetections.py $LSST/test_data --id patch=1,1 tract=0 filter=HSC-I^HSC-R --rerun example2:example3